## Attempting to select and split the groups 

In [1]:
# STEP 1: Must be done before anything each time notebook is loaded
# Import PySplit


import pysplit as py
import numpy as np
import pickle as pickle
from pprint import pprint
import os
from __future__ import print_function

In [2]:
# STEP 2: INITIALISE TRAJECTORY GROUP
# Group trajectories and then cycle through a group to determine distance, vector and rain status.
# check out traj_trajgroup_basics_examle.py

# Initialise Trajectories and Create a Trajectory Group
XI_traj=[]
XI_traj = py.make_trajectorygroup(r'/Volumes/Seagate_Backup/hysplit4/pysplitgen/*summer*')
# Calculate distance and vectors
for traj in XI_traj:
    traj.calculate_distance()
    traj.calculate_vector()
print('Total trajectories in group: ',XI_traj.trajcount)

Total trajectories in group:  166


Set the conditions for rainy conditions, the group must include rainy days.

In [3]:
rainylist = []

for traj in XI_traj:
    traj.set_rainstatus()
    if traj.rainy:
        rainylist.append(traj)
        
rainy_XI_traj = py.TrajectoryGroup(rainylist)
print('Total trajectories in rainy group: ',rainy_XI_traj.trajcount)
pprint(vars(rainy_XI_traj))

Total trajectories in rainy group:  79
{'trajcount': 79,
 'trajectories': [<pysplit.traj.Trajectory object at 0x11f314dd8>,
 'trajids': ['/Volumes/Seagate_Backup/hysplit4/pysplitgen/XI_long_testjan1500summer20060104041.0',
             '/Volumes/Seagate_Backup/hysplit4/pysplitgen/XI_long_testjan1500summer20060109041.0',
             '/Volumes/Seagate_Backup/hysplit4/pysplitgen/XI_long_testjan1500summer20060119041.0',
             '/Volumes/Seagate_Backup/hysplit4/pysplitgen/XI_long_testjan1500summer20060120041.0',
             '/Volumes/Seagate_Backup/hysplit4/pysplitgen/XI_long_testjan1500summer20060122041.0',
             '/Volumes/Seagate_Backup/hysplit4/pysplitgen/XI_long_testjan1500summer20060123041.0',
             '/Volumes/Seagate_Backup/hysplit4/pysplitgen/XI_long_testjan1500summer20060124041.0',
             '/Volumes/Seagate_Backup/hysplit4/pysplitgen/XI_long_testjan1500summer20060125041.0',
             '/Volumes/Seagate_Backup/hysplit4/pysplitgen/XI_long_testjan1500summer2

Creating different groups - Rainy, dry and all trajectories

In [9]:
dry_XI_trajgroup= XI_traj - rainy_XI_traj

print(dry_XI_trajgroup.trajcount)
print(XI_traj.trajcount)
print(rainy_XI_traj.trajcount)

87
166
79


In [5]:
moistureup = []

for traj in rainy_XI_traj:
    traj.moisture_uptake(-0.01,0.01,interval=6, vlim='prs')
    moistureup.append(traj)
    

In [6]:
pprint(vars(moistureup[2].uptake))

{'_data': BlockManager
Items: Index(['DateTime', 'Timestep', 'Cumulative_Dist', 'Avg_Pressure',
       'Avg_MixDepth', 'q', 'dq_initial', 'dq', 'above', 'below',
       'unknown_total', 'above_total', 'below_total', 'geometry'],
      dtype='object')
Axis 1: Float64Index([   0.0,   -6.0,  -12.0,  -18.0,  -24.0,  -30.0,  -36.0,  -42.0,
               -48.0,  -54.0,  -60.0,  -66.0,  -72.0,  -78.0,  -84.0,  -90.0,
               -96.0, -102.0, -108.0, -114.0, -120.0, -126.0, -132.0, -138.0,
              -144.0, -150.0, -156.0, -162.0, -168.0],
             dtype='float64', name='Timestep')
FloatBlock: [1, 2, 3, 4, 5, 10, 11, 12], 8 x 29, dtype: float64
ObjectBlock: [0, 6, 7, 8, 9, 13], 6 x 29, dtype: object,
 '_is_copy': None,
 '_item_cache': {},
 '_sindex': None,
 '_sindex_generated': False,
 'crs': None}


In [11]:
# STEP 3A: CALCULATE INTEGRATION ERRORS
# Calculate integration errors using reverse trajectories
# check out integration_error.py

c=[]
for traj in rainy_XI_traj[:]:
    traj.load_reversetraj()
    traj.calculate_integrationerr()
    c+=1

Trajectory /Volumes/Seagate_Backup/hysplit4/pysplitgen/XI_long_testjan1500summer20060104041.0 has a bad reverse trajectory: 
	
                          168 hours instead of 76 hours
Integration error calculation skipped for
                  Trajectory /Volumes/Seagate_Backup/hysplit4/pysplitgen/XI_long_testjan1500summer20060104041.0


TypeError: 'int' object is not iterable

In [123]:
# DEBUGGING STEP FOR IF MOISTURE UPTAKE AND HYPATH FAILS:
# FIND DISRUPTIVE PATH, REMOVE IT, RETURN TO STEP 2

for traj in rainy_XI_traj[:]:
    print('Error at trajectory',c,'of',len(list(enumerate(XI_traj))))
    pprint (vars(XI_traj[c]))

Error at trajectory 94 of 179
{'cfilename': 'XI_long_testjan1500summer2007010604CLIPPED',
 'cfolder': '/Volumes/Seagate_Backup/hysplit4/pysplitgen/clippedtraj',
 'cfullpath': '/Volumes/Seagate_Backup/hysplit4/pysplitgen/clippedtraj/XI_long_testjan1500summer2007010604CLIPPED',
 'data':           Timestep  Pressure  Rainfall  Mixing_Depth  Relative_Humidity  \
Timestep                                                                  
 0.0           0.0     849.9       0.0         937.4               86.8   
-1.0          -1.0     851.3       0.0         937.0               87.0   
-2.0          -2.0     852.7       0.0         933.3               87.0   
-3.0          -3.0     853.9       0.0         915.9               86.8   
-4.0          -4.0     855.0       0.0         894.1               86.7   
-5.0          -5.0     855.8       0.3         857.0               86.5   
-6.0          -6.0     856.7       0.3         833.6               86.4   
-7.0          -7.0     857.6       0.3 

 'multitraj': False,
 'parcel_num': 1.0,
 'path': <shapely.geometry.linestring.LineString object at 0x1174d5c50>,
 'trajcolor': 'black',
 'trajid': '/Volumes/Seagate_Backup/hysplit4/pysplitgen/XI_long_testjan1500summer20070106041.0'}
Error at trajectory 94 of 179
{'cfilename': 'XI_long_testjan1500summer2007010604CLIPPED',
 'cfolder': '/Volumes/Seagate_Backup/hysplit4/pysplitgen/clippedtraj',
 'cfullpath': '/Volumes/Seagate_Backup/hysplit4/pysplitgen/clippedtraj/XI_long_testjan1500summer2007010604CLIPPED',
 'data':           Timestep  Pressure  Rainfall  Mixing_Depth  Relative_Humidity  \
Timestep                                                                  
 0.0           0.0     849.9       0.0         937.4               86.8   
-1.0          -1.0     851.3       0.0         937.0               87.0   
-2.0          -2.0     852.7       0.0         933.3               87.0   
-3.0          -3.0     853.9       0.0         915.9               86.8   
-4.0          -4.0     855.0  

 'data':           Timestep  Pressure  Rainfall  Mixing_Depth  Relative_Humidity  \
Timestep                                                                  
 0.0           0.0     849.9       0.0         937.4               86.8   
-1.0          -1.0     851.3       0.0         937.0               87.0   
-2.0          -2.0     852.7       0.0         933.3               87.0   
-3.0          -3.0     853.9       0.0         915.9               86.8   
-4.0          -4.0     855.0       0.0         894.1               86.7   
-5.0          -5.0     855.8       0.3         857.0               86.5   
-6.0          -6.0     856.7       0.3         833.6               86.4   
-7.0          -7.0     857.6       0.3         822.4               86.4   
-8.0          -8.0     858.6       0.3         822.0               86.6   
-9.0          -9.0     859.7       0.2         830.9               86.8   
-10.0        -10.0     860.7       0.2         847.7               87.2   
-11.0        -11

 'data':           Timestep  Pressure  Rainfall  Mixing_Depth  Relative_Humidity  \
Timestep                                                                  
 0.0           0.0     849.9       0.0         937.4               86.8   
-1.0          -1.0     851.3       0.0         937.0               87.0   
-2.0          -2.0     852.7       0.0         933.3               87.0   
-3.0          -3.0     853.9       0.0         915.9               86.8   
-4.0          -4.0     855.0       0.0         894.1               86.7   
-5.0          -5.0     855.8       0.3         857.0               86.5   
-6.0          -6.0     856.7       0.3         833.6               86.4   
-7.0          -7.0     857.6       0.3         822.4               86.4   
-8.0          -8.0     858.6       0.3         822.0               86.6   
-9.0          -9.0     859.7       0.2         830.9               86.8   
-10.0        -10.0     860.7       0.2         847.7               87.2   
-11.0        -11

 'data':           Timestep  Pressure  Rainfall  Mixing_Depth  Relative_Humidity  \
Timestep                                                                  
 0.0           0.0     849.9       0.0         937.4               86.8   
-1.0          -1.0     851.3       0.0         937.0               87.0   
-2.0          -2.0     852.7       0.0         933.3               87.0   
-3.0          -3.0     853.9       0.0         915.9               86.8   
-4.0          -4.0     855.0       0.0         894.1               86.7   
-5.0          -5.0     855.8       0.3         857.0               86.5   
-6.0          -6.0     856.7       0.3         833.6               86.4   
-7.0          -7.0     857.6       0.3         822.4               86.4   
-8.0          -8.0     858.6       0.3         822.0               86.6   
-9.0          -9.0     859.7       0.2         830.9               86.8   
-10.0        -10.0     860.7       0.2         847.7               87.2   
-11.0        -11

...

{'cfilename': 'XI_long_testjan1500summer2007010604CLIPPED',
 'cfolder': '/Volumes/Seagate_Backup/hysplit4/pysplitgen/clippedtraj',
 'cfullpath': '/Volumes/Seagate_Backup/hysplit4/pysplitgen/clippedtraj/XI_long_testjan1500summer2007010604CLIPPED',
 'data':           Timestep  Pressure  Rainfall  Mixing_Depth  Relative_Humidity  \
Timestep                                                                  
 0.0           0.0     849.9       0.0         937.4               86.8   
-1.0          -1.0     851.3       0.0         937.0               87.0   
-2.0          -2.0     852.7       0.0         933.3               87.0   
-3.0          -3.0     853.9       0.0         915.9               86.8   
-4.0          -4.0     855.0       0.0         894.1               86.7   
-5.0          -5.0     855.8       0.3         857.0               86.5   
-6.0          -6.0     856.7       0.3         833.6               86.4   
-7.0          -7.0     857.6       0.3         822.4               86

{'cfilename': 'XI_long_testjan1500summer2007010604CLIPPED',
 'cfolder': '/Volumes/Seagate_Backup/hysplit4/pysplitgen/clippedtraj',
 'cfullpath': '/Volumes/Seagate_Backup/hysplit4/pysplitgen/clippedtraj/XI_long_testjan1500summer2007010604CLIPPED',
 'data':           Timestep  Pressure  Rainfall  Mixing_Depth  Relative_Humidity  \
Timestep                                                                  
 0.0           0.0     849.9       0.0         937.4               86.8   
-1.0          -1.0     851.3       0.0         937.0               87.0   
-2.0          -2.0     852.7       0.0         933.3               87.0   
-3.0          -3.0     853.9       0.0         915.9               86.8   
-4.0          -4.0     855.0       0.0         894.1               86.7   
-5.0          -5.0     855.8       0.3         857.0               86.5   
-6.0          -6.0     856.7       0.3         833.6               86.4   
-7.0          -7.0     857.6       0.3         822.4               86

{'cfilename': 'XI_long_testjan1500summer2007010604CLIPPED',
 'cfolder': '/Volumes/Seagate_Backup/hysplit4/pysplitgen/clippedtraj',
 'cfullpath': '/Volumes/Seagate_Backup/hysplit4/pysplitgen/clippedtraj/XI_long_testjan1500summer2007010604CLIPPED',
 'data':           Timestep  Pressure  Rainfall  Mixing_Depth  Relative_Humidity  \
Timestep                                                                  
 0.0           0.0     849.9       0.0         937.4               86.8   
-1.0          -1.0     851.3       0.0         937.0               87.0   
-2.0          -2.0     852.7       0.0         933.3               87.0   
-3.0          -3.0     853.9       0.0         915.9               86.8   
-4.0          -4.0     855.0       0.0         894.1               86.7   
-5.0          -5.0     855.8       0.3         857.0               86.5   
-6.0          -6.0     856.7       0.3         833.6               86.4   
-7.0          -7.0     857.6       0.3         822.4               86

{'cfilename': 'XI_long_testjan1500summer2007010604CLIPPED',
 'cfolder': '/Volumes/Seagate_Backup/hysplit4/pysplitgen/clippedtraj',
 'cfullpath': '/Volumes/Seagate_Backup/hysplit4/pysplitgen/clippedtraj/XI_long_testjan1500summer2007010604CLIPPED',
 'data':           Timestep  Pressure  Rainfall  Mixing_Depth  Relative_Humidity  \
Timestep                                                                  
 0.0           0.0     849.9       0.0         937.4               86.8   
-1.0          -1.0     851.3       0.0         937.0               87.0   
-2.0          -2.0     852.7       0.0         933.3               87.0   
-3.0          -3.0     853.9       0.0         915.9               86.8   
-4.0          -4.0     855.0       0.0         894.1               86.7   
-5.0          -5.0     855.8       0.3         857.0               86.5   
-6.0          -6.0     856.7       0.3         833.6               86.4   
-7.0          -7.0     857.6       0.3         822.4               86

{'cfilename': 'XI_long_testjan1500summer2007010604CLIPPED',
 'cfolder': '/Volumes/Seagate_Backup/hysplit4/pysplitgen/clippedtraj',
 'cfullpath': '/Volumes/Seagate_Backup/hysplit4/pysplitgen/clippedtraj/XI_long_testjan1500summer2007010604CLIPPED',
 'data':           Timestep  Pressure  Rainfall  Mixing_Depth  Relative_Humidity  \
Timestep                                                                  
 0.0           0.0     849.9       0.0         937.4               86.8   
-1.0          -1.0     851.3       0.0         937.0               87.0   
-2.0          -2.0     852.7       0.0         933.3               87.0   
-3.0          -3.0     853.9       0.0         915.9               86.8   
-4.0          -4.0     855.0       0.0         894.1               86.7   
-5.0          -5.0     855.8       0.3         857.0               86.5   
-6.0          -6.0     856.7       0.3         833.6               86.4   
-7.0          -7.0     857.6       0.3         822.4               86

In [124]:
print (XI_traj[c].fullpath)
print (XI_traj[c].filename)
filename=XI_traj[c].filename
path2= r'/Volumes/Seagate_Backup/hysplit4/pysplitgen/trajerrors'

/Volumes/Seagate_Backup/hysplit4/pysplitgen/XI_long_testjan1500summer2007010604
XI_long_testjan1500summer2007010604


In [125]:
os.rename(XI_traj[c].fullpath,
          os.path.join(path2,filename))